# Pivot and Save train_series.parquet

This Notebook will walk you through how we preprocessed the really long train_series.parquet

In [6]:
import dask.dataframe as dd
import os
import time
data_parquet = os.path.join('data','train_series.parquet')
print(f"Reading {data_parquet}")
t=time.time()
# data: dd.DataFrame = dd.read_parquet(data_parquet, columns=['step', 'series_id', 'anglez', 'enmo'], categories=['series_id'])
data: dd.DataFrame = dd.read_parquet(data_parquet)#, columns=['step', 'series_id', 'anglez', 'enmo'], categories=['series_id'])
print(f"Read {len(data)} rows from {data_parquet} in {time.time()-t:.3f} seconds")

Reading data\train_series.parquet
Read 127946340 rows from data\train_series.parquet in 15.336 seconds


In [7]:
data

,series_id,step,timestamp,anglez,enmo
npartitions=1,,,,,
,object,uint32,object,float32,float32
,...,...,...,...,...


## Attempt to save individual series files
I'm calling this attempt failed cuz its taking too long to save all the series files.
I will try to set up a process to process all the files over night though

In [8]:
sids = data['series_id'].unique().compute()

In [9]:
dir_path = os.path.join('data', 'parsed')
if not os.path.exists(dir_path):
    os.makedirs(dir_path)


In [15]:
sdone = os.listdir(dir_path)
sdone = set([i[:-8] for i in sdone if os.path.exists(os.path.join(dir_path, i, 'done.txt'))])
sids = [i for i in sids if i not in sdone]

In [16]:
sids[:10], f'total length = {len(sids)}'

(['785c9ca4eff7',
  'cfeb11428dd7',
  '9ddd40f2cb36',
  'b84960841a75',
  '6a4cd123bd69',
  'ca732a3c37f7',
  '51fdcc8d9fe7',
  '939932f1822d',
  'ea0770830757',
  '390b487231ce'],
 'total length = 250')

In [12]:
grouped_df = data.groupby('series_id')

In [13]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [21]:
import importlib
import parr
from itertools import repeat
importlib.reload(parr)
dfs = [grouped_df.get_group(i) for i in sids]
parr.parallelize(parr.save_series_id_to_parquet, dfs, sids, repeat(dir_path), total=len(sids))
# parr.parallelize(parr.hello2, repeat('hello',10), total=10) 

using 6 cores


  0%|          | 0/250 [05:05<?, ?it/s]


In [20]:
import multiprocessing
multiprocessing.cpu_count()

12

In [8]:
import tqdm

with(open('log.txt', 'w')) as f:
    f.write('')
for sid in tqdm.tqdm(sids):
    df = grouped_df.get_group(sid)
    dd.to_parquet(df, os.path.join(dir_path, f'{sid}.parquet'), write_index=False)
    with(open('log.txt', 'a')) as f:
        print(sid, file=f)
    

 10%|███▏                             | 27/277 [7:12:35<66:45:33, 961.33s/it]


KeyboardInterrupt: 

In [12]:
with(open('log.txt', 'r')) as f:
    lines = f.readlines()
    lines = set([i[:-1] for i in lines])

In [16]:
sids = set([i for i in sids if i not in lines])

In [17]:
len(sids)

250

In [40]:
dd.to_parquet?

Signature:
dd.to_parquet(
    df,
    path,
    engine='auto',
    compression='snappy',
    write_index=True,
    append=False,
    overwrite=False,
    ignore_divisions=False,
    partition_on=None,
    storage_options=None,
    custom_metadata=None,
    write_metadata_file=None,
    compute=True,
    compute_kwargs=None,
    schema='infer',
    name_function=None,
    filesystem=None,
    **kwargs,
)
Docstring:
Store Dask.dataframe to Parquet files

Notes
-----
Each partition will be written to a separate file.

Parameters
----------
df : dask.dataframe.DataFrame
path : string or pathlib.Path
    Destination directory for data.  Prepend with protocol like ``s3://``
    or ``hdfs://`` for remote data.
engine : {'auto', 'pyarrow', 'fastparquet'}, default 'auto'
    Parquet library to use. Defaults to 'auto', which uses ``pyarrow`` if
    it is installed, and falls back to ``fastparquet`` otherwise.
compression : string or dict, default 'snappy'
    Either a string like ``"snappy"`` or

## Attempt to pivot and save
kinda failed cus the process needed too much memory

In [ ]:
data['series_id'] = data['series_id'].cat.as_known()

In [ ]:
pivoted = dd.reshape.pivot_table(data, index='step', columns='series_id', values=['anglez','enmo'])

In [ ]:
pivoted

In [ ]:
anglez = pivoted['anglez']
anglez.columns = anglez.columns.astype('string')
anglez

In [ ]:
anglez.head()

In [ ]:
anglez.tail()

In [ ]:
enmo = pivoted['enmo']
enmo.columns = enmo.columns.astype('string')
enmo

In [ ]:
dir_path = os.path.join('data', 'parsed')
if not os.path.exists(dir_path):
    os.makedirs(dir_path)


In [ ]:
dd.to_parquet(enmo, os.path.join(*dir_path, 'enmo.parquet'))

In [ ]:
dd.to_parquet(anglez, os.path.join(*dir_path, 'anglez.parquet'))